## Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image

import os

### Create directory to save generated images

In [2]:
os.makedirs("gan_images", exist_ok=True)

## Define the Generator

In [3]:
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 28*28),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), 1, 28, 28)
        return img


# Define the Discriminator

In [4]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity


## Initialize Models and Optimizers

### Hyperparameters

In [5]:
latent_dim = 150
lr = 0.0002
batch_size = 64
n_epochs = 70

### Initialize models

In [6]:
generator = Generator(latent_dim)
discriminator = Discriminator()

### Optimizers

In [7]:
optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

### Loss function

In [8]:
adversarial_loss = nn.BCELoss()

### Check for CUDA

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator.to(device)
discriminator.to(device)
adversarial_loss.to(device)

BCELoss()

## Load the MNIST Dataset

In [10]:
dataloader = DataLoader(
    datasets.MNIST(
        "./data/mnist", train=True, download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5])
        ])
    ),
    batch_size=batch_size, shuffle=True
)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 11616786.24it/s]


Extracting ./data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 346612.00it/s]


Extracting ./data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 3203990.97it/s]


Extracting ./data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4709648.64it/s]

Extracting ./data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw



## Training loop

In [11]:
for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = torch.ones(imgs.size(0), 1, device=device, requires_grad=False)
        fake = torch.zeros(imgs.size(0), 1, device=device, requires_grad=False)

        # Configure input
        real_imgs = imgs.to(device)

        # -----------------
        #  Train Generator
        # -----------------
        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = torch.randn(imgs.size(0), latent_dim, device=device)

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        # Backprop and optimize
        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        # Backprop and optimize
        d_loss.backward()
        optimizer_D.step()

        # Print progress
        if i % 100 == 0:
            print(f"[Epoch {epoch}/{n_epochs}] [Batch {i}/{len(dataloader)}] " \
                  f"[D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f}]")

    # Save generated images every epoch
    save_image(gen_imgs.data[:25], f"gan_images/{epoch}.png", nrow=5, normalize=True)


[Epoch 0/70] [Batch 0/938] [D loss: 0.6889] [G loss: 0.6627]
[Epoch 0/70] [Batch 100/938] [D loss: 0.3352] [G loss: 1.0011]
[Epoch 0/70] [Batch 200/938] [D loss: 0.5301] [G loss: 0.7996]
[Epoch 0/70] [Batch 300/938] [D loss: 0.5558] [G loss: 0.9461]
[Epoch 0/70] [Batch 400/938] [D loss: 0.4089] [G loss: 0.9666]
[Epoch 0/70] [Batch 500/938] [D loss: 0.4014] [G loss: 0.9066]
[Epoch 0/70] [Batch 600/938] [D loss: 0.4180] [G loss: 0.9108]
[Epoch 0/70] [Batch 700/938] [D loss: 0.7227] [G loss: 2.5288]
[Epoch 0/70] [Batch 800/938] [D loss: 0.4996] [G loss: 0.8333]
[Epoch 0/70] [Batch 900/938] [D loss: 0.4625] [G loss: 1.0352]
[Epoch 1/70] [Batch 0/938] [D loss: 0.3608] [G loss: 1.1718]
[Epoch 1/70] [Batch 100/938] [D loss: 0.4698] [G loss: 0.5990]
[Epoch 1/70] [Batch 200/938] [D loss: 0.3886] [G loss: 1.2262]
[Epoch 1/70] [Batch 300/938] [D loss: 0.4225] [G loss: 0.7467]
[Epoch 1/70] [Batch 400/938] [D loss: 0.4213] [G loss: 1.6008]
[Epoch 1/70] [Batch 500/938] [D loss: 0.3810] [G loss: 2.06